In [ ]:
import os
import re
import json
from tqdm import tqdm
from utils.call_gpt import call_gpt, call_gpt_async, read_table_file

五大关系: depend, exclude, refer_to, only_include, combine

refer_to 条款引用
 - 源MEU需要结合目标条款的信息作为补充, 才能完整解释. 注意: "某某角色/情况的认定参考某某法律法规文件/法条", "处于某某法律法规文件/法条规定的的特殊情形", "参考附件2", "具体要求见附件4"这种需要参考其他文件/法条来判断的, 属于refer_to; "后续处理应当按照《公司法》、中国证监会和本所的相关规定办理"这种外部的整个法律的遵守也属于refer_to. 
 - 如果是"应当遵守某某法条", "在某情况下不适用/免于遵守某某法条", "只需要遵守某某法条", 这些在本法律法规文件内部进行遵守/免于遵守的关系, 分别属于should_include, exclude和only_include, 而非refer_to
 - 公司章程也可以被refer_to
 - 这是在生成MEU的函数时候起作用. Coding Agent需要去查找refer_to的目标条款
  
exclude 规则排除
 - 源MEU成立时(源MEU的主体subject符合, 且条件condition符合时), 使目标MEU失效. 
 - 关键词: 不受前款限制, 免于遵守xxx, 
 - 这是在MEU的函数执行后, 计算整个图的违规与否时起作用. 此时所有的MEU已经有这些结果: [主体适用/不适用, 条件符合/不符合, 违规/不违规], 此时找到exclude关系, 对于exclude的source, 如果其主体适用, 条件符合, 那么其target的MEU就标注为免除
  
only_include 仅适用
 - 源MEU成立时(源MEU的主体subject符合, 且条件condition符合时), 只需要考虑目标MEU的情况, 而不再需要考虑本法律法规文件内的任何其他的MEU. 
 - 这是在MEU的函数执行后, 计算整个图的违规与否时起作用. 
  
should_include 强制纳入
 - 当MEU_n_k中出现 {{"应当符合/遵循本指引第m, n, k条的要求", "应当参照/按照/参见本指引第m, n, k条处理",}} 时, 就是当前的MEU_n_k对Law_x, ..., Law_z存在should_include关系. 
 - should_include 的特点是可以免除一部分目标MEU的condition, 例如要约回购的MEU要求主体遵循某个condition为竞价回购的MEU的constrain, 此时应当直接检查constrain, 忽视condition的冲突. 
 - 只有明确声明应当遵循本法律法规内哪些法条的, 才是should_include关系. 指向目标不在本法律法规文件内部的, 以及未明确说明的应当遵循哪些法条的不予考虑. 注意: "某某情况的认定参考某某法律文件"的属于refer_to关系; "只需要考虑/遵守/按照某某要求办理"的属于only_include; 源MEU成立时可以免除目标MEU的属于exclude关系. 这些都不属于only_include关系. 
 - 这是在MEU的函数执行后, 计算整个图的违规与否时起作用. 强制纳入有时候会出现condition为"要约回购"的案例也符合condition为"竞价回购"的MEU的要求, 因此需要有针对性地生成新的强制纳入MEU, 或者考虑免除一些condition. 这里需要更多的例子. 

# prompt模板

In [2]:
prompt_get_relation = """
# MEU关系识别指令 **RELATION**

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的**RELATION**关系

## MEU概念简述
MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

## 核心任务
从给定的MEU列表中识别 **RELATION** 关系，输出(source_id, **RELATION**, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, **RELATION**, [target_id_1, target_id_2])
当所给的MEU之间或者MEU与法律之间不存在**RELATION**时, 如实返回空值, 不需要自己杜撰或强行拼凑**RELATION**. MEU间不存在关系是普遍的现象. 

## 注意事项
- 源MEU需要结合目标条款才能完整解释. 当**RELATION**的目标为某个具体MEU时候直接记录其编号MEU_n_k. 当**RELATION**的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
- 示例：
data:
[
    ...
]
relation: 
    ...


## 识别原则
1. 不修改MEU内容，仅建立关联
2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("MEU_n_i", "only_include", "MEU_n_j"),
("Law_n", "only_include", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的MEU列表:
{MEU_list}
"""

## refer_to

In [ ]:
prompt_get_refer_to = """
# MEU关系识别指令 refer_to (条款引用)

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的refer_to (条款引用)关系

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份") 
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

## 核心任务
从给定的MEU列表中识别 refer_to (条款引用) 关系，输出(source_id, refer_to, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, refer_to, [target_id_1, target_id_2])
当所给的MEU之间或者MEU与法律之间不存在refer_to时, 如实返回空值, 不需要自己杜撰或强行拼凑refer_to. MEU间不存在refer_to关系是普遍的现象.  

## refer_to关系的含义与注意事项
- refer_to关系的含义是: "某某角色/情况的认定参考某某法律法规文件/法条", "处于某某法律法规文件/法条规定的的特殊情形", "参考附件2", "具体要求见附件4"这种需要参考其他文件/法条来判断的, 属于refer_to; "后续处理应当按照《公司法》、中国证监会和本所的相关规定办理"这种外部的整个法律的遵守也属于refer_to.
- 如果是"应当遵守某某法条", "在某情况下不适用/免于遵守某某法条", "只需要遵守某某法条", 这些在本法律法规文件内部进行遵守/免于遵守的关系, 分别属于should_include, exclude和only_include, 而非refer_to
- 当refer_to的目标为某个具体MEU时候直接记录其编号MEU_n_k; 当refer_to的目标为本法(或者"本指引")的第a条法条时, 编号为"Law_a". 当refer_to的目标不在本法之间的东西时候, 直接记录改目标为字符串.  
- 示例：
data:
[
    {{"MEU_id":"MEU_24_1","subject":"大股东 | 一致行动人","condition":"","constraint":"应当共同遵守本指引关于大股东减持股份的规定","contextual_info":""}},
    {{"MEU_id":"MEU_24_2","subject":"","condition":"","constraint":"","contextual_info":"一致行动人的认定适用《上市公司收购管理办法》规定"}}
]
relation: 
    ("MEU_12_2", "refer_to", "一致行动人的认定适用《上市公司收购管理办法》规定")


## 更多经验
 - 先根据关键词进行寻找, 再根据经验和逻辑进行筛选
 - "本指引", "本办法"和"本法"等自指的问题不需要记录"refer_to", 但"参考本指引第n条"这种具体的自指需要(source, refer_to Law_n)
 - refer_to的对象为某个外部法律时, 请直接采用("source_id", "refer_to", "目标法律的字符串"), 不得自己新建Law节点
 - 立法纲领性条款, 例如"依托aaa法, bbb法和ccc法指定本法"的, 不需要构建refer_to关系, 例如"依据《中华人民共和国公司法》《中华人民共和国证券法》《北京证券交易所上市公司持续监管办法（试行）》《上市公司独立董事管理办法》《北京证券交易所股票上市规则（试行）》制定本指引"不需要建立refer_to关系. 
 - 当MEU中有"应聘请独立财务顾问寻求帮助"和"应按照证监会的要求"等target_id不为某法律, 某法条或某MEU的, 不需要记录refer_to关系. 
 - 一些关键词: 前款, 前述等. 如果遇到"前款"内容被拆分到其他MEU, 需要进行refer_to; 如果"前款"涉及的MEU特别多, 也可以直接refer_to整个法条Law_n
 - 我们的整个系统另下有其他agent负责管理"only_include"关系, 你不需要处理. 当你看到"主体S_1在情况Cd_1下, **只需要考虑**本法第m, n, k条"这种表述时, 就是典型的"only_include"关系, 不需要添加refer_to关系. 
 - 我们有其他agent负责处理"should_include"关系, 你不需要处理. 当你看到"主体S_1在情况Cd_1下, **应当遵守/应当按照...办理/应当符合**本法第m, n, k条"这种表述时, 就是典型的"only_include"关系. 更多例子: "应当在减持后6个月内继续遵守本指引第四条规定", ""
 - 当且仅当某MEU_n_i中提及"请结合本法规第m条第k款"这种表述时, 你可以设立一个"Law_n_k", 并声称("MEU_n_i", "refer_to", "Law_n_k"). 你可以这么做的原因是MEU与法条的款没有直接对应关系, 为了忠于原文的准确表述可以用符号记录法条的款的信息. 我们会在后处理中解析Law_n_k格式的法条的款的信息, 所以不用担心. 
 - refer_to 有时候可能是一对多的关系, 而且MEU的subject, condition, constrain和contextual_info都有可能存在条款引用, 请仔细检查避免遗漏. 


## 识别原则
1. 不修改MEU内容，仅建立关联
2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("MEU_n_i", "refer_to", "MEU_n_j"),
("Law_n", "refer_to", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的MEU列表:
{MEU_list}
"""

## combine

In [ ]:
# prompt_get_combine = """
# # MEU关系识别指令 combine (需合并检验)

# ## 角色定位
# 你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的combine (需合并检验)关系

# ## MEU概念简述
# MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
# - MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
# - subject: 责任主体（如"控股股东"）
# - condition: 触发条件（如"减持股份"） 
# - constraint: 约束内容（如"提前15日公告"）
# - contextual_info: 补充说明（如价格计算方式）
# 我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

# ## 核心任务
# 从给定的MEU列表中识别 combine 关系，输出(source_id, combine, target_id)三元组. 
# 当target_id有多个时可以用列表承载, 例如(source_id, combine, [target_id_1, target_id_2])
# 当所给的MEU之间或者MEU与法律之间不存在combine时, 如实返回空值, 不需要自己杜撰或强行拼凑combine. MEU间不存在关系是普遍的现象. 

# ## combine关系的含义与注意事项
# - combine关系的含义: 当且仅当若干个MEU其实在讲述同一个数值的计算方法, 且实际上无法分开独立检验时, 应该采用combine关系. 当MEU可以独立检验的时候, 禁止合并计算, 因为这与MEU的原子性要求所违背. 
# - 注意, combine是不需要区分方向的关系, 当遇到A, B和C等多个MEU或法条需要combine时, 只需要("A", "combine", ["B", "C"]), 不需要另外记录B combine AC和C combine AB的关系. 
# - 当combine的目标为某个具体MEU时候直接记录其编号MEU_n_k. 当combine的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
# - 注意, 只有当若干MEU无法独立检验时, 才符合combine关系. 有时候MEU只是简单的并列关系, 例如"主体S在条件Cd下受到约束Cs1"和"主体S在条件Cd下受到约束Cs2"只是简单的并列关系, 可以独立执行检验, 不需要合并. 
# - 接上条, 需要combine的实际上通常是一些不完整的, 不好归类的MEU. 完整的MEU呈现并列等结构不需要进行combine. 
# - 示例：
# data:[
#     {{"MEU_id":"MEU_18_1","subject":"上市公司董监高","condition":"计算可转让股份数量时","constraint":null,"contextual_info":"以上年末所持股份总数作为基数计算可转让股份数量"}},
#     {{"MEU_id":"MEU_18_2","subject":"上市公司董监高","condition":"当年存在可转让但未转让的本公司股份时","constraint":"应当将未转让股份计入当年末所持股份总数","contextual_info":"该总数将作为次年可转让股份的计算基数"}},
#     {{"MEU_id":"MEU_18_3","subject":"上市公司董监高","condition":"所持本公司股份在年内增加且新增无限售条件股份时","constraint":"新增无限售条件股份当年可转让25%","contextual_info":null}},
#     {{"MEU_id":"MEU_18_4","subject":"上市公司董监高","condition":"所持本公司股份在年内增加且新增有限售条件股份时","constraint":"新增有限售条件股份应当计入次年可转让股份的计算基数","contextual_info":null}},
#     {{"MEU_id":"MEU_18_5","subject":"上市公司董监高","condition":"因上市公司实施权益分派导致所持本公司股份增加时","constraint":null,"contextual_info":"可同比例增加当年可转让数量"}}
# ]
# relation: 
#     ("MEU_18_1", "combine", ["MEU_18_2", "MEU_18_3", "MEU_18_4", "MEU_18_5"])

# ## 更多经验
#  - combine是一个很少见的关系. 当你觉得你发现了combine你需要仔细想想: 这几个MEU/Law真的完全无法独立检查, 必须要拼凑在一起吗? combine通常只会在一些不良性划分的MEU之间出现. 
#  - 当若干个MEU在讲述同一个披露的方案/公告中应当包含的不同内容时, 不需要进行combine, 因为其可以独立执行, 只需各自独立检查公告中是否包含所需内容即可. example: [上市公司, 制定股份回购方案, 应当包含股份回购方式] 和 [上市公司, 制定股份回购方案, 应当包含股份回购的资金总额和金额来源]
#  - 当有若干个MEU分别提及应当披露若干个不同的公告时, 不需要进行combine, 因为我们可以单独地检查每一个公告是否被披露, 这些MEU可以单独分别执行. 
#  - 当一个MEU讲述"应当披露某某公告", 而另有一些MEU讲述"该公告应当包含某某内容"时, 不需要进行combine, 因为这些公告都可以独立执行, 第一个MEU可以通过检查公司是否依归披露公告来执行, 第二个MEU可以通过检查该MEU中是否存在该内容来执行. 
#  - 当若干MEU_n_i, MEU_n_j, MEU_n_k描述一个连续的流程时, 不需要进行combine. 我们希望找到的是在检验过程不合并就无法执行的条目, 而非在法律规定的流程中必须合并的条目. 

# ## 识别原则
# 1. 不修改MEU内容，仅建立关联
# 2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
# 3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

# ## 输出格式
# 用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
# <RELATIONS>
# [
# ("MEU_n_i", "combine", "MEU_n_j"),
# ("Law_n", "combine", "Law_m")
# ]
# </RELATIONS>


# 接下来是等待你发掘关系的MEU列表:
# {MEU_list}
# """

In [ ]:
"""
我们在研究llm驱动的合规检查, 提出了MEU这个概念: 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份") 
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

现在, 我们觉得MEU这个名字不够好, 因为:
1. 我们并没有拆分到字面意思的"最小", 而是拆分到差不多能用就结束了. 因为我们是用LLM来拆解MEU, 也是用LLM来执行MEU的操作
2. 感觉"执行"的意思有点奇怪
3. 感觉和法律/金融/会计等领域的合规没什么特别直接的关系. 不一定要字面意思上有关系, 但理论的背景上希望能找到这些领域的理论来论述我们这样设置这几个结构的合理性

请你帮我设计一个新的名字, 解决上面的问题
"""

## depend

In [ ]:
# prompt_get_depend = """
# # MEU关系识别指令 depend（检查过程中的依赖）

# ## 角色定位
# 你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的depend（检查过程中的依赖）关系

# ## MEU概念简述
# MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
# - MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
# - subject: 责任主体（如"控股股东"）
# - condition: 触发条件（如"减持股份"） 
# - constraint: 约束内容（如"提前15日公告"）
# - contextual_info: 补充说明（如价格计算方式）
# 我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

# ## 核心任务
# 从给定的MEU列表中识别 depend 关系，输出(source_id, depend, target_id)三元组. 
# 当target_id有多个时可以用列表承载, 例如(source_id, depend, [target_id_1, target_id_2])
# 当所给的MEU之间或者MEU与法律之间不存在depend时, 如实返回空值, 不需要自己杜撰或强行拼凑depend. MEU间不存在关系是普遍的现象. 

# ## depend关系的含义与注意事项
# - depend关系的含义是: 在我们进行合规检查时, 目标MEU的执行必须以源MEU完成为前提. depend关系通常只在涉及指标的计算的时候出现, 例如, MEU_n_i表示应当按照某种方法计算某个指标, 而MEU_n_j要求该指标必须处在特定的范围之内, 那么就必须先执行MEU_n_i, 后执行MEU_n_j, 记为("MEU_n_i", "depend", "MEU_n_j"). 
# - 注意: 我们定义的是在合规检查时存在的depend关系, 与办事流程中的先后顺序不同, 请注意鉴别. 你应当带着这样的思考去寻找depend关系: 我想要检验这些MEU的时候, 是否需要其他MEU的结论作为前提? 你不应当带着这样的思想: 我想要执行这个MEU的时候, 是否在法律规定的办事流程中存在依赖? 
# - 当depend的目标为某个具体MEU时候直接记录其编号MEU_n_k. 当depend的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
# - 示例：
# data: 
# [
#     {{"MEU_id":"MEU_17_1","subject":"上市公司董监高","condition":"在就任时确定的任期内和任期届满后6个月内通过集中竞价、大宗交易、协议转让等方式转让股份且非因司法强制执行、继承、遗赠、依法分割财产等导致股份变动","constraint":"每年转让的股份不得超过其所持本公司股份总数的25%","contextual_info":""}},
#     {{"MEU_id":"MEU_18_1","subject":"上市公司董监高","condition":"计算可转让股份数量时","constraint":"","contextual_info":"以上年末所持股份总数作为基数计算可转让股份数量"}}
# ]
# relation: 
#     ("MEU_17_1", "depend", "MEU_18_1")


# ## 更多经验
#  - 我们在设计depend关系时, 关注的核查过程中是否存在依赖关系, 某一MEU的结论的判断需要另一MEU的结果作为中间数据, 而非在流程的进行过程中某一MEU需要在另一MEU的后面执行. 例如, 减持指引条例的Law_18讲述股东可转让股份的基数如何计算, 而MEU_17_1提到每年转让的股份不得超过基数的25%, 那么存在(MEU_17_1, depend, Law_18)的关系, 我们要先执行Law18及其下属单元得到可转让股份的基数, 才能执行MEU_17_1, 借助该基数判断是否违规. 
#  - 容易混淆的关系: 在规定的办事流程中, 某一MEU在另一MEU之后进行的, 不属于depend关系, 因为这是办事流程的depend, 不是我们检查流程中的depend. 例如, 某企业应当严格管理内幕消息知情人, 某企业应当及时登记内幕消息知情人信息, 在办事流程上前者是后者的基础, 但在我们的检查流程中可以分别检查并给出定性判断. 又例如, 
#  - 容易混淆的关系: 某一MEU规定了需要披露某文件, 另一MEU规定了该文件需要包含的内容的, 不属于depend关系, 因为这两个MEU完全可以分开执行, 执行第一个MEU时检查是否披露了该文件, 制定第二个MEU时检查是否披露了文件, 如披露, 其中是否包含应有的内容. 
#  - 在整个系统中, 有另外的agent负责寻找"refer_to"关系, 含义是源MEU需要结合目标条款才能完整解释, 例如某MEU_n_i提及"在本指引第七条和第六条完成后课召开董事会商讨转板事宜", 或者某MEU_n_j提及"根据本指引第十七条进行判断", 这些都属于条款引用的refer_to关系, 而非你需要找寻的depend关系. 你不用理会这些条款. 
#  - depend关系并不是一个常见的关系, 当你觉得发现了depend关系, 请认真思考你发现的是否真的是我们定义的"在核查过程中某一MEU需要依赖于另一MEU的运行结果". 

 
# ## 识别原则
# 1. 不修改MEU内容，仅建立关联
# 2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
# 3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

# ## 输出格式
# 用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
# <RELATIONS>
# [
# ("MEU_n_i", "depend", "MEU_n_j"),
# ("Law_n", "depend", "Law_m")
# ]
# </RELATIONS>


# 接下来是等待你发掘关系的MEU列表:
# {MEU_list}
# """

## exclude

In [6]:
prompt_get_exclude = """
# MEU关系识别指令 exclude（规则排除）

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的exclude（规则排除）关系

## MEU概念简述
MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

## 核心任务
从给定的MEU列表中识别 exclude 关系，输出(source_id, exclude, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, exclude, [target_id_1, target_id_2])
当所给的MEU之间或者MEU与法律之间不存在exclude时, 如实返回空值, 不需要自己杜撰或强行拼凑exclude. MEU间不存在关系是普遍的现象. 

## exclude关系的含义与注意事项
- exclude关系的含义是: 源MEU成立时(源MEU的主体subject符合, 且条件condition符合时), 使目标MEU失效. 例如, MEU_n_i表示主体S_1在情况Cd_1下应当遵守约束Cs_1, 而MEU_n_j则声明当主体S_1在情况Cd_2时可以不遵循前款约束Cs_1, 那么就可以理解为MEU_n_j对MEU_n_i进行了规则排除, 记为("MEU_n_j", "exclude", "MEU_n_i")
- 当exclude的目标为某个具体MEU时候直接记录其编号MEU_n_k. 当exclude的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
- 示例：
data: 
[
    {{"MEU_id":"MEU_17_1","subject":"上市公司董监高","condition":"在就任时确定的任期内和任期届满后6个月内通过集中竞价、大宗交易、协议转让等方式转让股份且非因司法强制执行、继承、遗赠、依法分割财产等导致股份变动","constraint":"每年转让的股份不得超过其所持本公司股份总数的25%","contextual_info":""}},
    {{"MEU_id":"MEU_17_2","subject":"上市公司董监高","condition":"所持股份不超过1000股","constraint":"可一次全部转让且不受前款转让比例限制","contextual_info":""}}
]
relation: 
    ("MEU_17_2", "exclude", "MEU_17_1")

## 更多经验
 - 一般来说只有明确出现"可以不受前款限制", "无需遵守第m, n和k条法律"的才是exclude关系. 
 - 有些MEU的condition项存在如"因离婚等原因减持股份, 且不属于证监会规定的除外情况"表述, 这里的"除外"可能导致望文生义, 但其实与我们探讨的exclude没有什么关系. 请你回归定义去理解. 
 - 有时一簇MEU存在若干类似的表述, 例如"上市公司, 仅改变募投项目实施地点, 应由董事会审议通过, 免于在股东大会上进行审议"和"上市公司, 改变募集资金用途, 应由董事会和股东大会审议通过", 前者存在"免于..."的表述, 可能导致望文生义, 但是应当注意前者和后者的condition是不同的, 前者仅改变募投项目实施地点, 后者要改变募集资金用途. 
 - 有些MEU存在"主体S_1在情况Cd_1下需要遵守限制Cs_1"和"主体S_2在情况Cd_2无需遵守限制Cs_1"的表述, 这里的"无需遵守"可能导致望文生义, 但需注意这里的subject和condition都是不同的, 他们之间也不存在exclude关系. 
 - 如果没有看到"可以不受前款/某某法条限制"等触发词, 请不要自己推理和杜撰exclude关系, 例如公司想进行某项活动, MEU_n_i规定需要股东大会审议通过, MEU_n_j规定需要董事会审议通过, 这两个要求是并列关系, 不能因为股东大会的等级比董事会高就认定MEU_n_i抹除了MEU_n_j. 

## 识别原则
1. 不修改MEU内容，仅建立关联
2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("MEU_n_i", "exclude", "MEU_n_j"),
("Law_n", "exclude", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的MEU列表:
{MEU_list}
"""

## only_include

In [7]:
prompt_get_only_include = """
# MEU关系识别指令 only_include (仅适用)

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的only_include (仅适用)关系

## MEU概念简述
MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

## 核心任务
从给定的MEU列表中识别 only_include 关系，输出(source_id, only_include, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, only_include, [target_id_1, target_id_2])
当所给的MEU之间或者MEU与法律之间不存在only_include时, 如实返回空值, 不需要自己杜撰或强行拼凑only_include. MEU间不存在关系是普遍的现象. 

## only_include关系的含义与注意事项
- only_include关系的含义是: 源MEU成立时(源MEU的主体subject符合, 且条件condition符合时), 只需要考虑目标MEU的情况, 而不再需要考虑本法律法规文件内的任何其他的MEU. 如果没有排他性, 则不构成only_include关系. 例如, MEU_n_i表示主体S_1在情况Cd_1下, 只需要考虑本法第m, n, k条, 这就是视为MEU_n_i仅包含其所列示的几条, 记为("MEU_n_j", "only_include", ["Law_m", "Law_n", "Law_k"])
- 当only_include的目标为某个具体MEU时候直接记录其编号MEU_n_k. 当only_include的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
- 示例：
data:
[
    {{"MEU_id":"MEU_7_1","subject":"上市公司大股东","condition":"减持通过本所和全国中小企业股份转让系统的竞价或做市交易买入的本公司股份","constraint":"只适用本指引第二条、第三条、第十条、第十一条、第二十三条、第二十七条的规定","contextual_info":""}}
]
relation: 
    ("MEU_7_1", "only_include", ["Law_2", "Law_3", "Law_10", "Law_11", "Law_23", "Law_27"])

    
## 更多经验
 - 通常只有当遇到"主体S在情况Cd下仅适用第i, j和k法条(而不再需要适用本法律的任何其他法条)"这种表述出发, 才会触发;only_include. 如果没有排他性, 而是"适用若干条款"的表述, 不是only_include关系. 
 - 在整个工作流程中, 有其他的agent帮忙寻找exclude关系, 这个关系大意是当source的主体和条件触发以后, 可以免除target的考核评估. 你如果发现exclude可以不予理会, 会有其他agent处理这种关系. 
 - 我们有其他的agent帮忙寻找refer_to关系, 当某个MEU称"参考某某法律文件"或者"参考本法律法规的第m条进行认定", 就是refer_to关系. 你不需要处理refer_to关系. 
 - 我们有其他的agent负责寻找should_include关系, 这个关系是指当且仅当某MEU明确声明强制纳入本法律法规内哪些法条的, 例如"应当符合/遵循/参照/按照本指引第x, ..., z条的规定...". 这个关系有应当遵守的意思, 但并非是"仅遵守". 你不需要处理should_include关系. should_include关系是没有排他性的, 而你需要负责的only_include是有排他性的, 也即不再需要遵守目标MEU以外的其他任何MEU. 
 - only_include是一个很少见的关系. 当你觉得你发现了only_include你需要仔细想想: 这个MEU的意思真是可以排除本法条的其他一切吗? 


## 识别原则
1. 不修改MEU内容，仅建立关联
2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("MEU_n_i", "only_include", "MEU_n_j"),
("Law_n", "only_include", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的MEU列表:
{MEU_list}
"""

## should_include

In [8]:
prompt_get_should_include = """
# MEU关系识别指令 should_include (强制纳入)

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 MEU (法律的最小可执行单元) 中的should_include (强制纳入)关系

## MEU概念简述
MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用MEU判断案例的合规性时, 会先检查案例主体是否符合MEU主体, 再检查案例中的条件是否符合MEU中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了MEU中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该MEU上违规, 否则会判定该案例在该MEU上不违规. 

## 核心任务
从给定的MEU列表中识别 should_include 关系，输出(source_id, should_include, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, should_include, [target_id_1, target_id_2])
当所给的MEU之间或者MEU与法律之间不存在should_include时, 如实返回空值, 不需要自己杜撰或强行拼凑should_include. MEU间不存在关系是普遍的现象. 

## should_include关系的含义与注意事项
- should_include关系的定义: 当MEU_n_k中出现 {{"应当符合/遵循本指引第m, n, k条的要求", "应当参照/按照/参见本指引第m, n, k条处理",}} 时, 就是当前的MEU_n_k对Law_x, ..., Law_z存在should_include关系. 
- 只有明确声明强制纳入本法律法规内哪些法条的, 才是should_include关系. 指向目标不在本法律法规文件内部的, 以及未明确说明的强制纳入哪些法条的不予考虑. 例如"一致行动人的认定参照本所上市准则", 其莫表不再本法律文件(本法/本指引)之内, 而且这是一个refer_to关系, 你不需要考虑. 
- 示例：
data:
[
    {{"MEU_id":"MEU_50_1","subject":"上市公司","condition":"实施要约回购","constraint":"应当符合本指引第十三条至第十五条、第十七条、第十九条、第二十条的规定","contextual_info":""}}
]
relation: 
    ("MEU_7_1", "should_include", ["Law_13", "Law_14", "Law_15", "Law_17", "Law_29", "Law_20"])

    
## 更多经验
 - 我们有另外的agent处理refer_to关系, 这与你负责的should_include容易混淆. refer_to关系是指, 源MEU需要结合目标条款才能完整解释. 常见的refer_to表述有: "应当共同遵守本指引关于大股东减持股份的规定", "一致行动人的认定适用《上市公司收购管理办法》规定". 这几种情况没有明确声明强制纳入本法律法规内哪些法条的规定, 因此你都不需要考虑建立should_include关系. 
 - 我们有另外的agent负责管理"only_include"关系, 你不需要处理. 当你看到"主体S_1在情况Cd_1下, 只需要考虑本法第m, n, k条"这种表述时, 就是典型的"only_include"关系, 你不需要再在这些MEU之间添加should_include关系. 
 - 我们有其他的agent帮忙寻找refer_to关系, 当某个MEU称"参考某某法律文件"或者"参考本法律法规的第m条进行认定", 就是refer_to关系. 你不需要处理refer_to关系. 你只需要处理明确的""


## 识别原则
1. 不修改MEU内容，仅建立关联
2. 允许MEU之间建立关系, 让MEU与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("MEU_n_i", "should_include", "MEU_n_j"),
("Law_n", "should_include", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的MEU列表:
{MEU_list}
"""

# 批量调用LLM获取回复

In [9]:
import pandas as pd

# 定义文件路径和要提取的列
file_path = "law_to_MEU/st_3_0_MEU/GT/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv"
columns_to_extract = ["MEU_id", "subject", "condition", "constraint", "contextual_info"]

# 读取CSV文件
df = pd.read_csv(file_path)

# 提取所需的列
extracted_df = df[columns_to_extract]

# 将提取的数据转换为Python列表
MEU_list = extracted_df.to_dict(orient="records")

# from pprint import pprint
# # 打印结果
# pprint(MEU_list_example)

MEU_list[:3]

[{'MEU_id': 'MEU_1_1',
  'subject': nan,
  'condition': nan,
  'constraint': nan,
  'contextual_info': '为规范北京证券交易所上市公司股东及董监高持股管理和减持股份行为，维护市场秩序，保护投资者合法权益，根据《上市公司股东减持股份管理暂行办法》《北京证券交易所上市公司持续监管办法（试行）》《上市公司董事、监事和高级管理人员所持本公司股份及其变动管理规则》《北京证券交易所股票上市规则（试行）》等规定制定本指引'},
 {'MEU_id': 'MEU_2_1',
  'subject': '上市公司股东 | 董监高',
  'condition': '减持其持有的本公司股份',
  'constraint': '应当遵守法律法规、部门规章、本所业务规则及本指引的相关规定',
  'contextual_info': nan},
 {'MEU_id': 'MEU_2_2',
  'subject': '上市公司股东 | 董监高',
  'condition': '对持股比例、持股数量、持股期限、减持方式、减持比例、减持价格等做出公开承诺',
  'constraint': '应当严格履行所做出的承诺',
  'contextual_info': nan}]

In [ ]:
prompt_list = [
    prompt_get_refer_to,
    # prompt_get_combine,
    # prompt_get_depend,
    prompt_get_exclude,
    prompt_get_only_include,
]

prompt_0 = prompt_get_refer_to.format(MEU_list=MEU_list)

## 异步获取回复

In [ ]:
import asyncio
from tqdm.asyncio import tqdm_asyncio
from utils.call_gpt import call_gpt_async
import csv
import os

async def process_MEU_relations_async(
    file_name, 
    prompt_list,
    max_concurrency=5,
):
    """
    异步处理函数
    
    参数：
    prompt_list: 提示词模板列表
    MEU_list: 用于格式化的MEU列表
    max_concurrency: 最大并行请求数（默认5）
    """

    if file_name.endswith(".csv"):
        pass
    else:
        file_name += ".csv"

    input_dir = r"law_to_MEU/st_3_0_MEU/GT" 
    output_dir=r"law_to_MEU/st_4_MEU_relations/raw_response"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)
    columns_to_extract = ["MEU_id", "subject", "condition", "constraint", "contextual_info"]

    # 读取CSV文件
    df = pd.read_csv(input_path)
    extracted_df = df[columns_to_extract]
    MEU_list = extracted_df.to_dict(orient="records")

    # 共享状态
    results = []
    failed_prompts = []
    semaphore = asyncio.Semaphore(max_concurrency)

    async def process_single(prompt_template):
        """处理单个提示词的异步任务"""
        async with semaphore:
            try:
                formatted_prompt = prompt_template.format(MEU_list=MEU_list)
                
                content, reasoning_content, api_usage = await call_gpt_async(
                    prompt=formatted_prompt,
                    api_key="35684824-1776-48b6-94fd-96c2e99d0724",
                    base_url="https://ark.cn-beijing.volces.com/api/v3",
                    model="ep-20250217153824-9xcbx",
                    # temperature=0.6,
                )

                return {
                    "response": content,
                    "reasoning_content": reasoning_content,
                    "api_usage": api_usage
                }
            except Exception as e:
                failed_prompts.append(str(prompt_template))
                return None

    # 创建所有任务
    tasks = [process_single(prompt) for prompt in prompt_list]
    
    # 执行并带进度条
    pbar = tqdm_asyncio(total=len(tasks), desc="Processing prompts")
    
    for future in asyncio.as_completed(tasks):
        result = await future
        if result:
            results.append(result)
        pbar.update()

    pbar.close()

    # 保存结果到单个CSV文件
    with open(output_path, "w", encoding='utf-8-sig', newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["response", "reasoning_content", "api_usage"])
        writer.writeheader()
        writer.writerows(results)

    # 打印简要报告
    print(f"\nSuccess: {len(results)}")
    print(f"Failed: {len(failed_prompts)}")

    return failed_prompts

In [12]:
# failed_prompts = await process_MEU_relations_async(
#     prompt_list=[
#         prompt_get_refer_to,
#         prompt_get_combine,
#         prompt_get_depend,
#         prompt_get_exclude,
#         prompt_get_only_include,
#     ],
#     file_name = '北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理', 
#     max_concurrency=5
# )

## 解析回复结果

In [13]:
import csv
import re
import json
import os

def get_relations_from_responses(file_name):
    if file_name.endswith(".csv"):
        pass
    else:
        file_name += ".csv"

    input_dir = r"law_to_MEU/st_4_MEU_relations/raw_response"
    output_dir = r"law_to_MEU/st_4_MEU_relations"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)

    def clean_response(response):
        """清理和解析response列中的关系数据"""
        try:
            # 统一处理转义符号
            response = response.replace("<\\RELATIONS>", "</RELATIONS>")
            response = response.replace("<\\\\RELATIONS>", "</RELATIONS>")
            
            # 提取所有RELATIONS标签内容
            matches = re.findall(r'<RELATIONS>(.*?)</RELATIONS>', response, re.DOTALL)
            
            if matches:
                # 选择内容最长的匹配项
                longest_content = max(matches, key=lambda x: len(x.strip())).strip()
                
                # 数据格式转换
                longest_content = longest_content.replace('""', '"')  # 处理双引号转义
                longest_content = longest_content.replace('(', '[').replace(')', ']')  # 转换括号格式
                
                # 解析为JSON数组
                return json.loads(longest_content)
            return []
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
            print(f"问题内容: {longest_content}")
            return []
        except Exception as e:
            print(f"数据处理异常: {e}")
            return []

    # 读取原始CSV文件
    with open(input_path, mode='r', encoding='utf-8-sig') as infile:
        reader = csv.DictReader(infile)
        law_articles = list(reader)

    # 处理关系数据
    split_data = []
    fieldnames = ['source', 'relation', 'target']
    # print(law_articles[0])
    
    for row in law_articles:
        relations = clean_response(row['response'])
        for rel_entry in relations:
            # 校验数据格式
            if not isinstance(rel_entry, list) or len(rel_entry) != 3:
                print(f"异常条目: {rel_entry}")
                continue
            
            source, relation, targets = rel_entry
            # 处理target为列表的情况
            if isinstance(targets, list):
                for target in targets:
                    split_data.append({
                        'source': source.strip('"'),  # 去除可能的残留引号
                        'relation': relation.strip('"'),
                        'target': target.strip('"')
                    })
            else:
                split_data.append({
                    'source': source.strip('"'),
                    'relation': relation.strip('"'),
                    'target': targets.strip('"')
                })

    # 写入处理后的CSV文件
    with open(output_path, mode='w', encoding='utf-8-sig', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(split_data)
    
    print(f'文件已保存至: {output_path}')

In [14]:
# 使用示例
# get_relations_from_responses("北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv")

## 主workflow

In [ ]:
prompt_list=[
        prompt_get_refer_to,
        # prompt_get_combine,
        # prompt_get_depend,
        prompt_get_exclude,
        prompt_get_only_include,
        prompt_get_should_include,
    ]

# 定义文件目录路径
directory_path = r"law_to_MEU/st_3_0_MEU/GT"

# 提取所有.doc文件的文件名，不包含路径
filenames = [
    f for f in os.listdir(directory_path) if f.endswith('.csv')
]

filenames 

['北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv',
 '北京证券交易所上市公司持续监管指引第5号——要约收购.csv',
 '北京证券交易所上市公司持续监管指引第1号——独立董事.csv',
 '北京证券交易所上市公司持续监管指引第4号——股份回购.csv',
 '北京证券交易所上市公司持续监管指引第10号——权益分派.csv']

In [16]:

for filename in filenames:
    failed_prompts = await process_MEU_relations_async(
    prompt_list=prompt_list,
    file_name = filename, 
    max_concurrency=5
    )
    get_relations_from_responses(filename)



Processing prompts: 100%|██████████| 6/6 [03:06<00:00, 31.12s/it]



Success: 6
Failed: 0
文件已保存至: law_to_MEU/st_4_MEU_relations/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv


Processing prompts: 100%|██████████| 6/6 [00:57<00:00,  9.55s/it]



Success: 6
Failed: 0
文件已保存至: law_to_MEU/st_4_MEU_relations/北京证券交易所上市公司持续监管指引第5号——要约收购.csv


Processing prompts: 100%|██████████| 6/6 [00:47<00:00,  7.97s/it]



Success: 6
Failed: 0
文件已保存至: law_to_MEU/st_4_MEU_relations/北京证券交易所上市公司持续监管指引第1号——独立董事.csv


Processing prompts: 100%|██████████| 6/6 [04:42<00:00, 47.15s/it]



Success: 6
Failed: 0
文件已保存至: law_to_MEU/st_4_MEU_relations/北京证券交易所上市公司持续监管指引第4号——股份回购.csv


Processing prompts: 100%|██████████| 6/6 [01:10<00:00, 11.82s/it]


Success: 6
Failed: 0
文件已保存至: law_to_MEU/st_4_MEU_relations/北京证券交易所上市公司持续监管指引第10号——权益分派.csv


# temp

In [17]:
# relations_str = """
# [
# ("MEU_24_2", "refer_to", "一致行动人的认定适用《上市公司收购管理办法》规定"),
# ("MEU_21_2", "refer_to", "本指引关于协议转让方式减持股份的规定"),
# ("MEU_19_1", "refer_to", "上市公司章程"),
# ("MEU_18_1", "combine", ["MEU_18_2", "MEU_18_3", "MEU_18_4", "MEU_18_5"]),
# ("MEU_17_1", "depend", "LAW_18"),
# ("MEU_3_4", "depend", "MEU_3_3"),
# ("MEU_25_8", "depend", "MEU_25_7"),
# ("MEU_5_5", "depend", "MEU_4_2"),
# ("MEU_17_2", "exclude", "MEU_17_1"),
# ("MEU_4_9", "exclude", "MEU_4_2"),
# ("MEU_7_1", "only_include", ["LAW_2", "LAW_3", "LAW_10", "LAW_11", "LAW_23", "LAW_27"]),
# ("MEU_7_2", "only_include", ["LAW_2", "LAW_3", "LAW_10", "LAW_11", "LAW_12", "LAW_23", "LAW_27"]),
# ("MEU_13_3", "refer_to", "LAW_12"),
# ("MEU_15_1", "refer_to", "本所《上市规则》第十章"),
# ("MEU_15_2", "refer_to", "本所《上市规则》第十章"),
# ("MEU_22_6", "refer_to", "本指引关于协议转让方式减持股份的规定"),
# ("MEU_12_2", "refer_to", "LAW_12"),
# ("MEU_12_3", "refer_to", "LAW_12"),
# ("MEU_24_6", "refer_to", ["LAW_11", "LAW_12"]),
# ("MEU_4_10", "refer_to", "LAW_12")
# ]
# """

In [18]:
# print(prompt_get_relation.format(MEU_list=MEU_list_example))

# MEU_list = [
    
# ]

# prompt = prompt_get_relation




In [19]:
# prompt_get_relation = """
# # MEU关系识别指令

# ## 角色定义
# 资深法律逻辑分析专家，擅长从最小可执行单元（MEU）中发现规则间的逻辑关联

# ## 核心任务
# 从给定的MEU列表中识别三种基础关系，输出(source_id, relation_type, target_id)三元组. 
# 当target_id有多个时可以用列表承载, 例如(source_id, relation_type, [target_id_1, target_id_2])
# 当所给的MEU之间或者MEU与法律之间不存在relation时, 如实返回空值, 不需要自己杜撰或强行拼凑relation. MEU间不存在关系是普遍的现象. 

# ## MEU概念简述
# MEU（Minimum Executable Unit）是法律条文拆解出的最小合规单元，包含：
# - MEU_id: MEU的编号, 通常为"MEU_a_b", 其中a是其所属的法条的编号, b是其在法条内部的编号
# - subject: 责任主体（如"控股股东"）
# - condition: 触发条件（如"减持股份"） 
# - constraint: 约束内容（如"提前15日公告"）
# - contextual_info: 补充说明（如价格计算方式）

# ## 关系类型定义

# ### 1. refer_to（条款引用） 
# - 特征：源MEU需要结合目标条款才能完整解释. 当refer_to的目标为某个具体MEU时候直接记录其编号. 当refer_to的目标为本法(或者"本指引")的第a条法条时, 编号为"LAW_a". 当refer_to的目标不在本法之间的东西时候, 直接记录改目标为字符串.  
# - 示例：
# data:
# [
#     {{"MEU_id":"MEU_24_1","subject":"大股东 | 一致行动人","condition":"","constraint":"应当共同遵守本指引关于大股东减持股份的规定","contextual_info":""}},
#     {{"MEU_id":"MEU_24_2","subject":"","condition":"","constraint":"","contextual_info":"一致行动人的认定适用《上市公司收购管理办法》规定"}}
# ]
# relation: 
#     ("MEU_12_2", "refer_to", "一致行动人的认定适用《上市公司收购管理办法》规定")

# ### 2. combine
# - 特征：当且仅当若干个MEU其实在讲述同一个数值的计算方法, 且实际上无法分开独立执行时, 应该采用combine关系. 当MEU可以可以独立执行的时候, 禁止合并计算, 因为这与MEU的原子性要求所违背. 注意, combine是不需要区分方向的关系, 当遇到A, B和C需要combine时, 只需要("A", "combine", ["B", "C"]), 不需要另外记录B combine AC和C combine AB的关系. 
# - 示例：
# data:[
#     {{"MEU_id":"MEU_18_1","subject":"上市公司董监高","condition":"计算可转让股份数量时","constraint":null,"contextual_info":"以上年末所持股份总数作为基数计算可转让股份数量"}},
#     {{"MEU_id":"MEU_18_2","subject":"上市公司董监高","condition":"当年存在可转让但未转让的本公司股份时","constraint":"应当将未转让股份计入当年末所持股份总数","contextual_info":"该总数将作为次年可转让股份的计算基数"}},
#     {{"MEU_id":"MEU_18_3","subject":"上市公司董监高","condition":"所持本公司股份在年内增加且新增无限售条件股份时","constraint":"新增无限售条件股份当年可转让25%","contextual_info":null}},
#     {{"MEU_id":"MEU_18_4","subject":"上市公司董监高","condition":"所持本公司股份在年内增加且新增有限售条件股份时","constraint":"新增有限售条件股份应当计入次年可转让股份的计算基数","contextual_info":null}},
#     {{"MEU_id":"MEU_18_5","subject":"上市公司董监高","condition":"因上市公司实施权益分派导致所持本公司股份增加时","constraint":null,"contextual_info":"可同比例增加当年可转让数量"}}
# ]
# relation: 
#     ("MEU_18_1", "combine", ["MEU_18_2", "MEU_18_3", "MEU_18_4", "MEU_18_5"])

    
# ### 3. depend（流程依赖）
# - 特征：目标MEU的执行必须以源MEU完成为前提
# - 示例：
# data: 
# [
#     {{"MEU_id":"MEU_17_1","subject":"上市公司董监高","condition":"在就任时确定的任期内和任期届满后6个月内通过集中竞价、大宗交易、协议转让等方式转让股份且非因司法强制执行、继承、遗赠、依法分割财产等导致股份变动","constraint":"每年转让的股份不得超过其所持本公司股份总数的25%","contextual_info":""}},
#     {{"MEU_id":"MEU_18_1","subject":"上市公司董监高","condition":"计算可转让股份数量时","constraint":"","contextual_info":"以上年末所持股份总数作为基数计算可转让股份数量"}}
# ]
# relation: 
#     ("MEU_17_1", "depend", "LAW_18")

# ### 4. exclude（规则排除）
# - 特征：源MEU成立时(源MEU的主体subject符合, 且条件condition符合时), 使目标MEU失效
# - 示例：
# data: 
# [
#     {{"MEU_id":"MEU_17_1","subject":"上市公司董监高","condition":"在就任时确定的任期内和任期届满后6个月内通过集中竞价、大宗交易、协议转让等方式转让股份且非因司法强制执行、继承、遗赠、依法分割财产等导致股份变动","constraint":"每年转让的股份不得超过其所持本公司股份总数的25%","contextual_info":""}},
#     {{"MEU_id":"MEU_17_2","subject":"上市公司董监高","condition":"所持股份不超过1000股","constraint":"可一次全部转让且不受前款转让比例限制","contextual_info":""}}
# ]
# relation: 
#     ("MEU_17_2", "exclude", "MEU_17_1")

    
# ### 5. only_include (仅适用)
# - 特征: "当主体A在条件B下仅适用本法第三条, 第四条和第七条". 虽然在逻辑上only_include和exclude可以互相转化, 但为整个系统运行简洁, 当法条中接近于"仅适用"时我们就采用only_include作为relation. 
# - 示例：
# data:
# [
#     {{"MEU_id":"MEU_7_1","subject":"上市公司大股东","condition":"减持通过本所和全国中小企业股份转让系统的竞价或做市交易买入的本公司股份","constraint":"只适用本指引第二条、第三条、第十条、第十一条、第二十三条、第二十七条的规定","contextual_info":""}}
# ]
# relation: 
#     ("MEU_7_1", "only_include", ["LAW_2", "LAW_3", "LAW_10", "LAW_11", "LAW_23", "LAW_27"])


# ## 识别原则
# 1. 原子性保护：不修改MEU内容，仅建立关联
# 2. 层级穿透：允许跨法条建立关系（如MEU_A排除Law_B）


# ## 输出格式
# 用<RELATIONS>标签包裹的 Python 列表：
# [
# ("MEU_X", "exclude", "MEU_Y"),
# ("MEU_A", "depend", "MEU_B")
# ]
# </RELATIONS>


# 接下来是等待你发掘关系的MEU列表:
# {MEU_list}
# """

In [20]:
# 我现在有一个csv, 其columns有[law_article_num,law_article,MEU_id,subject,condition,constraint,contextual_info,api_usage,reasoning_content], 其中我会用law_article_num,MEU_id来绘制一个graph, law节点连接到下属的MEU节点, MEU节点有这些属性[subject,condition,constraint,contextual_info]. 接下来我会引入relation这个概念, 我有一些这样的relation: <RELATIONS>
# [
# ("MEU_24_2", "refer_to", "一致行动人的认定适用《上市公司收购管理办法》规定"),
# ("MEU_21_2", "refer_to", "本指引关于协议转让方式减持股份的规定"),
# ("MEU_19_1", "refer_to", "上市公司章程"),
# ("MEU_18_1", "combine", ["MEU_18_2", "MEU_18_3", "MEU_18_4", "MEU_18_5"]),
# ("MEU_17_1", "depend", "LAW_18"),
# ("MEU_3_4", "depend", "MEU_3_3"),
# ("MEU_25_8", "depend", "MEU_25_7"),
# ("MEU_5_5", "depend", "MEU_4_2"),
# ("MEU_17_2", "exclude", "MEU_17_1"),
# ("MEU_4_9", "exclude", "MEU_4_2"),
# ("MEU_7_1", "only_include", ["LAW_2", "LAW_3", "LAW_10", "LAW_11", "LAW_23", "LAW_27"]),
# ("MEU_7_2", "only_include", ["LAW_2", "LAW_3", "LAW_10", "LAW_11", "LAW_12", "LAW_23", "LAW_27"]),
# ("MEU_13_3", "refer_to", "LAW_12"),
# ("MEU_15_1", "refer_to", "本所《上市规则》第十章"),
# ("MEU_15_2", "refer_to", "本所《上市规则》第十章"),
# ("MEU_22_6", "refer_to", "本指引关于协议转让方式减持股份的规定"),
# ("MEU_12_2", "refer_to", "LAW_12"),
# ("MEU_12_3", "refer_to", "LAW_12"),
# ("MEU_24_6", "refer_to", ["LAW_11", "LAW_12"]),
# ("MEU_4_10", "refer_to", "LAW_12")
# ]
# </RELATIONS>. 我希望你帮我写代码从这个字符串中把relation提取为csv, 然后帮我修改我的echart绘图代码, 加上relation. 我的原始代码如下: import pandas as pd
# from pyecharts import options as opts
# from pyecharts.charts import Graph
# from pyecharts.commons.utils import JsCode


# def gen_tree(df, filename):

#     # 创建节点和边
#     nodes = [{"name": "Root", "symbolSize": 30}]
#     edges = []

#     # 添加法条编号为一级节点，最小可执行单元编号为二级节点
#     for _, row in df.iterrows():
#         law_article_num = row['law_article_num']
#         MEU_id = row['MEU_id']
#         subject = row['subject']
#         condition = row['condition']
#         constraint = row['constraint']
#         contextual_info = row['contextual_info']  # 新增字段

#         # 为法条编号生成唯一节点名称
#         law_article_node = f"Law_{law_article_num}"

#         # 添加法条节点
#         if not any(node['name'] == law_article_node for node in nodes):
#             nodes.append({"name": law_article_node, "symbolSize": 20})
#             edges.append({"source": law_article_node, "target": "Root"})

#         # 添加MEU节点
#         nodes.append({
#             "name": MEU_id,
#             "symbolSize": 10,
#             "value": f"Subject: {subject}<br>Condition: {condition}<br>Constraint: {constraint}<br>contextual_info: {contextual_info}"
#         })
#         edges.append({"source": MEU_id, "target": law_article_node})

#     # 创建图表
#     graph = (
#         Graph(init_opts=opts.InitOpts(width="1200px", height="800px"))
#         .add(
#             "",
#             nodes,
#             edges,
#             repulsion=8000,
#             edge_symbol=['circle', 'arrow'],
#             tooltip_opts=opts.TooltipOpts(formatter=JsCode("""
#                 function(params) {
#                     return params.data.value;
#                 }
#             """))
#         )
#         .set_global_opts(title_opts=opts.TitleOpts(title="Law Articles and Minimum Executable Units"))
#     )

#     # 渲染图表
#     graph.render(f"{filename}.html")
